In [ ]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold


In [ ]:
"""
各種設定
"""
pd.set_option('display.max_columns', 100)
RANDOM_STATE = 2020

In [ ]:
"""
データ読み込み
train_X, train_y
test_X
"""
train = pd.read_csv("../../../cat-in-the-dat-ii/output/datas/train_linear_mini.csv", index_col=0).sort_index(inplace=False)
train_X = train.drop(['target'], axis=1)
train_y = train['target']
test_X = pd.read_csv("../../../cat-in-the-dat-ii/output/datas/test_linear_mini.csv", index_col=0).sort_index(inplace=False)
sample_submission = pd.read_csv("../../../cat-in-the-dat-ii/input/sample_submission.csv", index_col=0)

# 不要な変数を削除
del train


In [ ]:
"""
KFoldでvalidation
"""
# kf = KFold(n_splits=5, shuffle=False, random_state=RANDOM_STATE)
kf = StratifiedKFold(n_splits=5, shuffle=False, random_state=RANDOM_STATE)

EnsembleModels = []
RocScores = {}

for i, [tr_index, val_index] in tqdm(enumerate(kf.split(train_X, train_y))):
    model = Ridge(
        alpha=5,
        random_state=RANDOM_STATE,
    )
    
    TempModel = model.fit(train_X.iloc[tr_index, :], train_y.iloc[tr_index])
                                                # eval_set=[(train_X.iloc[val_index, :], train_y.iloc[val_index])],
                                                # verbose=1,
                                                # eval_metric='auc',
                                                # early_stopping_rounds=100)
    EnsembleModels.append(
        (f'Ridge_{i}', TempModel))
    
    # validation
    RocScores[f'{i}'] = [
        roc_auc_score(train_y.iloc[val_index], TempModel.predict(train_X.iloc[val_index, :]))
    ]


In [ ]:
"""
モデルを保存
"""
with open("../../../cat-in-the-dat-ii/output/models/RidgeRegression.pkl", mode="wb") as f:
    pickle.dump(EnsembleModels, file=f)
    print('モデル保存しました')

"""
ローカルCV結果を保存
"""
with open("../../../cat-in-the-dat-ii/output/results/RidgeRegression_localCV.pkl", mode="wb") as f:
    pickle.dump(RocScores, file=f)
    print('ローカルCVを保存しました')


In [ ]:
"""
テストテータの予測と保存
"""
# dataframe.append()  おっそーーー
# dict -> dataframe  はっやーーー
EnsembleDf = {}
for name_model in tqdm(EnsembleModels):
    temp = pd.Series(data=name_model[1].predict(test_X), index=test_X.index, name=name_model[0])
    EnsembleDf[name_model[0]] = temp

EnsembleDf = pd.DataFrame.from_dict(EnsembleDf, orient='columns')
EnsembleDf.set_index(keys=sample_submission.index[:], inplace=True)
sample_submission['target'] = EnsembleDf.mean(axis=1)
sample_submission.to_csv("../../../cat-in-the-dat-ii/output/datas/sample_submission_lgb.csv")
print('sample_submission_lgbを保存しました')